In [1]:
import gEconpy as ge
import gEconpy.plotting as gp

# gEconpy preliminaries

## GCN file

I've already written the model into a GCN file and saved it. Here is the file.

In [2]:
with open('rbc_log_utility.gcn', 'r') as file:
    print(file.read())

assumptions
{
	positive
	{
		K[], A[], mc[], r[], w[], Y[], alpha;
	};
};

block HOUSEHOLD
{
	definitions
	{
		u[] = gamma * log(C[]) + psi * log(L[]);
	};

	objective
	{
		U[] = u[] + beta * E[][U[1]];
	};

	controls
	{
		C[], I[], K[], L[], N[];
	};

	constraints
	{
		C[] + I[] = w[] * N[] + r[] * K[-1] + Div[] : lambda[];
		N[] + L[] = 1;
		K[] = (1 - delta) * K[-1] + I[];
	};

	calibration
	{
		beta = 0.98;
		delta = 0.035;
		gamma = 1;
		N[ss] = 0.33 -> psi;
	};
};

block FIRM
{
	objective
	{
		TC[] = -(w[] * N[] + r[] * K[-1]);
	};

	controls
	{
		N[], K[-1];
	};

	constraints
	{
		Y[] = A[] * K[-1] ^ alpha * N[] ^ (1 - alpha): mc[];
	};

	identities
	{
		mc[] = 1;
		Div[] = Y[] + TC[];
	};
	
	calibration
	{
		alpha = 0.33;
	};
};

block EXOGENOUS
{
	identities
	{
		log(A[]) = rho * log(A[-1]) + epsilon[];
	};

	shocks
	{
		epsilon[];
	};

	calibration
	{
		rho = 0.95;
	};
};



## Load the model

To better see how things work under the hood, I am disabling automatic symbolic simplification. This is never actually recommended.

In [3]:
mod_log = ge.gEconModel('rbc_log_utility.gcn', 
                    simplify_tryreduce=False, 
                    simplify_blocks=False,
                    simplify_constants=False)

Model Building Complete.
Found:
	16 equations
	16 variables
	1 stochastic shock
		 0 / 1 has a defined prior. 
	5 parameter
		 0 / 5 has a defined prior. 
	1 calibrating equation
	1 parameter to calibrate
 Model appears well defined and ready to proceed to solving.



# Questions

## Household First Order Conditions

gEconpy derives first order conditions automatically. We can access these block-by-block by looking into the `mod_logblocks` property. The first 3 equations are the model constraints, then the objective, then the derivatives of the block lagrangian, in the order they were entered by the user.

In [4]:
import sympy as sp
L = sp.Symbol('\mathcal L')
household_controls = mod_log.blocks['HOUSEHOLD'].controls
household_equations = mod_log.blocks['HOUSEHOLD'].system_equations
for control, eq in zip(household_controls, household_equations[-len(household_controls):]):
    display(sp.Eq(sp.diff(L, control, evaluate=False), eq))

Eq(Derivative(\mathcal L, C_t), gamma/C_t - lambda_t)

Eq(Derivative(\mathcal L, I_t), lambda__H_2_t - lambda_t)

Eq(Derivative(\mathcal L, K_t), beta*(-lambda__H_2_t+1*(delta - 1) + lambda_t+1*r_t+1) - lambda__H_2_t)

Eq(Derivative(\mathcal L, L_t), psi/L_t - lambda__H_1_t)

Eq(Derivative(\mathcal L, N_t), -lambda__H_1_t + lambda_t*w_t)

In [5]:
dL_dC, dL_dI, dL_dK, dL_dL, dL_dN = household_equations[4:]

To get the desired equations, we need to do a bit of manipulation. This is easy to do because all of these expressions are sympy expressions, allowing for fast symbolic solving/substitution. 

First, we need to add the model variables to the notebook global namespace so we can refer to them.

In [6]:
all_vars = [v for x in mod_log.variables for v in [x.step_backward(), x, x.step_forward(), x.to_ss()]]
var_dict = {}
for x in all_vars:
    var_dict[x.safe_name] = x
for x in mod_log.free_param_dict.to_sympy().keys():
    var_dict[x.name] = x
for x in mod_log.params_to_calibrate:
    var_dict[x.name] = x
    
globals().update(var_dict)

### Euler Equation

In [7]:
from gEconpy.shared.utilities import step_equation_forward, step_equation_backward, eq_to_ss
lam = sp.solve(dL_dC, lambda_t)[0]
lam2 = sp.solve(dL_dI.subs({lambda_t:lam}), lambda__H_2_t)[0]
euler = dL_dK.subs({lambda__H_2_t:lam2, 
                    lambda__H_2_tp1:step_equation_forward(lam2),
                    lambda_tp1:step_equation_forward(lam2)})
euler = sp.Eq(C_tp1 / C_t, sp.solve(euler, C_tp1)[0] / C_t)

In [8]:
euler

Eq(C_t+1/C_t, beta*(-delta + r_t+1 + 1))

### Labor supply curve

In [9]:
lam1 = sp.solve(dL_dN, lambda__H_1_t)[0]
wage = sp.solve(dL_dL.subs({lambda__H_1_t:lam1, lambda_t:lam}), w_t)[0]
sp.Eq(w_t, wage)

Eq(w_t, psi*C_t/(gamma*L_t))

## Firm FOC

In the exact same way, we can look into the firm FoC.

In [10]:
firm_controls = mod_log.blocks['FIRM'].controls
firm_equations = mod_log.blocks['FIRM'].system_equations
for control, eq in zip(firm_controls, firm_equations[-len(firm_controls):]):
    display(sp.Eq(L.diff(control, evaluate=False), eq))

Eq(Derivative(\mathcal L, N_t), A_t*K_t-1**alpha*mc_t*(1 - alpha)/N_t**alpha - w_t)

Eq(Derivative(\mathcal L, K_t-1), alpha*A_t*K_t-1**(alpha - 1)*N_t**(1 - alpha)*mc_t - r_t)

The only difference with the reference answers is the inclusion of the marginal cost term, which is normalized to 1 in perfect competition. 

In [11]:
sp.Eq(w_t, sp.solve(firm_equations[-2], w_t)[0])

Eq(w_t, A_t*K_t-1**alpha*mc_t*(1 - alpha)/N_t**alpha)

In [12]:
sp.Eq(r_t, sp.solve(firm_equations[-1], r_t)[0])

Eq(r_t, alpha*A_t*K_t-1**(alpha - 1)*N_t**(1 - alpha)*mc_t)

## Implicit Market Clearing

This is easily shown by substituting the definition of total costs into the definition dividends, then putting the resulting term into the household budget constraint. 

In [13]:
household_budget = household_equations[0]
dividends = sp.solve(firm_equations[1], Div_t)[0]
total_cost = sp.solve(firm_equations[3], TC_t)[0]

household_budget.subs({Div_t:dividends,
                       TC_t:total_cost}).simplify()

-C_t - I_t + Y_t

## Derive the steady state

Step 1: Get factor prices

In [14]:
# r_ss is easy, it fall straight out of the Euler equation.
r_ss_solved= sp.solve(eq_to_ss(euler), r_ss)[0]

# for w_ss, plug factor demands back into the production function,
# then solve for marginal cost
am1 = sp.Symbol(r'\alpha^{-1}', positive=True, real=True)
production_func = mod_log.blocks['FIRM'].system_equations[2]

capital_demand = sp.solve(firm_equations[-1]
     .replace(K_tm1 ** (alpha - 1), K_tm1 ** alpha / K_tm1)
     .subs(production_func + Y_t, Y_t), K_tm1)[0]

labor_demand = sp.solve(firm_equations[-2]
     .replace(N_t ** -alpha, N_t ** (1 - alpha) / N_t)
     .subs(production_func + Y_t, Y_t), N_t)[0]

mc_func = production_func.subs({K_tm1:capital_demand,
                      N_t:labor_demand,
                      A_t:1})

mc_ss_solved = eq_to_ss(sp.solve(mc_func.replace(Y_t, 1).replace(mc_t, 1) + 1 - (1 / mc_t), mc_t)[0])
w_ss_solved = sp.solve(1 - mc_ss_solved, w_ss)[0]

display(sp.Eq(r_ss, r_ss_solved))
display(sp.Eq(w_ss, w_ss_solved))

Eq(r_ss, delta - 1 + 1/beta)

Eq(w_ss, r_ss**(alpha/(alpha - 1))/(alpha**alpha*(1 - alpha)**(1 - alpha))**(1/(alpha - 1)))

Step 2: Clear factor markets

In [15]:
# Capital Market
capital_supply = eq_to_ss(household_equations[2])
I_ss_solved = sp.solve(capital_supply.simplify().subs({K_ss:eq_to_ss(capital_demand), mc_ss:1}), I_ss)[0]
sp.Eq(I_ss, I_ss_solved)

Eq(I_ss, alpha*delta*Y_ss/r_ss)

In [16]:
labor_demand

Y_t*mc_t*(1 - alpha)/w_t

In [17]:
# Labor market
labor_supply = w_ss - eq_to_ss(wage)
C_ss_solved = sp.solve(labor_supply.subs({L_ss: (1 - N_ss), N_ss:eq_to_ss(labor_demand), mc_ss:1}), C_ss)[0]
sp.Eq(C_ss, C_ss_solved)

Eq(C_ss, gamma*(alpha*Y_ss - Y_ss + w_ss)/psi)

Step 3: Clear AS-AD

In [18]:
Y_ss_solved = sp.solve(Y_ss - C_ss_solved - I_ss_solved, Y_ss)[0].subs({w_ss:w_ss_solved, r_ss:r_ss_solved}).subs({r_ss:r_ss_solved})
sp.Eq(Y_ss, Y_ss_solved)

Eq(Y_ss, -gamma*(delta - 1 + 1/beta)*(delta - 1 + 1/beta)**(alpha/(alpha - 1))/((alpha**alpha*(1 - alpha)**(1 - alpha))**(1/(alpha - 1))*(alpha*delta*psi + alpha*gamma*(delta - 1 + 1/beta) - gamma*(delta - 1 + 1/beta) - psi*(delta - 1 + 1/beta))))

Now that we have $Y_{ss}$ as a function of parameters only, we can go back and complete the whole system

In [19]:
C_ss_solved_2 = C_ss_solved.subs([(Y_ss, Y_ss_solved), (w_ss, w_ss_solved), (r_ss, r_ss_solved)]).simplify()
I_ss_solved_2 = I_ss_solved.subs([(Y_ss, Y_ss_solved), (r_ss, r_ss_solved)])
K_ss_solved_2 = sp.solve(capital_supply, K_ss)[0].subs([(I_ss, I_ss_solved_2), (Y_ss, Y_ss_solved)])
N_ss_solved_2 = eq_to_ss(labor_demand).subs([(mc_ss, 1), (Y_ss, Y_ss_solved), (w_ss, w_ss_solved), (r_ss, r_ss_solved)])
L_ss_solved_2 = 1 - eq_to_ss(labor_demand).subs([(mc_ss, 1), (Y_ss, Y_ss_solved), (w_ss, w_ss_solved), (r_ss, r_ss_solved)])

display(sp.Eq(C_ss, C_ss_solved_2))
display(sp.Eq(I_ss, I_ss_solved_2))
display(sp.Eq(K_ss, K_ss_solved_2))
display(sp.Eq(N_ss, N_ss_solved_2))
display(sp.Eq(L_ss, L_ss_solved_2))

Eq(C_ss, gamma*((beta*delta - beta + 1)/(alpha*beta))**(alpha/(alpha - 1))*(alpha*beta*delta - beta*delta + beta - 1)/(((1 - alpha)/(1 - alpha)**alpha)**(1/(alpha - 1))*(alpha*beta*delta*gamma + alpha*beta*delta*psi - alpha*beta*gamma + alpha*gamma - beta*delta*gamma - beta*delta*psi + beta*gamma + beta*psi - gamma - psi)))

Eq(I_ss, -alpha*delta*gamma*(delta - 1 + 1/beta)**(alpha/(alpha - 1))/((alpha**alpha*(1 - alpha)**(1 - alpha))**(1/(alpha - 1))*(alpha*delta*psi + alpha*gamma*(delta - 1 + 1/beta) - gamma*(delta - 1 + 1/beta) - psi*(delta - 1 + 1/beta))))

Eq(K_ss, -alpha*gamma*(delta - 1 + 1/beta)**(alpha/(alpha - 1))/((alpha**alpha*(1 - alpha)**(1 - alpha))**(1/(alpha - 1))*(alpha*delta*psi + alpha*gamma*(delta - 1 + 1/beta) - gamma*(delta - 1 + 1/beta) - psi*(delta - 1 + 1/beta))))

Eq(N_ss, -gamma*(1 - alpha)*(delta - 1 + 1/beta)/(alpha*delta*psi + alpha*gamma*(delta - 1 + 1/beta) - gamma*(delta - 1 + 1/beta) - psi*(delta - 1 + 1/beta)))

Eq(L_ss, gamma*(1 - alpha)*(delta - 1 + 1/beta)/(alpha*delta*psi + alpha*gamma*(delta - 1 + 1/beta) - gamma*(delta - 1 + 1/beta) - psi*(delta - 1 + 1/beta)) + 1)

Sympy gives an easy way to wrap these equations into a function we can evaluate for whatever parameter values we like

In [20]:
f_steady_state = sp.lambdify([alpha, beta, delta, gamma, psi],
                            [r_ss_solved,
                             w_ss_solved.subs([(r_ss, r_ss_solved)]),
                             Y_ss_solved,
                             C_ss_solved_2,
                             I_ss_solved_2,
                             K_ss_solved_2,
                             N_ss_solved_2])

# Put the output into a dictionary for easy reading
dict(zip([r_ss, w_ss, Y_ss, C_ss, I_ss, K_ss, N_ss], 
         f_steady_state(alpha=0.33, beta=0.98, delta=0.035, gamma=1, psi=1.719)))

{r_ss: 0.055408163265306176,
 w_ss: 1.6134732143531099,
 Y_ss: 0.7945524526013914,
 C_ss: 0.6289255794707245,
 I_ss: 0.1656268731306656,
 K_ss: 4.732196375161873,
 N_ss: 0.3299404901843179}

## Parameter Calibration 

Calibration of parameters can be done by 1) looking at micro studies, 2) targeting national accounting ratios, or 3) researching estimated posterior parameter values estimated in studies of the same or similar countries.

- $\alpha$ - 
- $\beta$ - financial models (consumption CAPM, for example), take $\frac{1}{\beta}$ to be the long-term interest rate. That's been about 3% historically in the US, so $\beta$ can be set to 1 / 1.03 = 0.97.
- $\delta$
- $\gamma$ - The utility funciton is over-parameterized. $\gamma$ can be set to 1.
- $\psi$ - Target 8 working hours per days, or $N_{ss} = \frac{8}{24} = \frac{1}{3}$
- $\rho_A$ - 
- $\sigma_A$ -

## Transversality Condition

Transversality is a modeling contrivence to ensure that agents can't borrow infinite money to get infinite consumption. If only it were actually possible to do :(

## Compute Steady State

Internally, `model.steady_state` uses a root finding algorithm to get the steady state. We see that it gives almost exactly the same result as the hand-computed steady state above. In the GCN file we calibrated $\psi$ to make $N_ss = 0.33$. We see that this condition has been satisfed at $\psi = 1.719$.

In [21]:
mod_log.steady_state()

Steady state found! Sum of squared residuals is 3.9651845564799144e-25


In [22]:
mod_log.print_steady_state()

A_ss                    1.000
C_ss                    0.629
Div_ss                 -0.000
I_ss                    0.166
K_ss                    4.733
L_ss                    0.670
N_ss                    0.330
TC_ss                  -0.795
U_ss                  -57.590
Y_ss                    0.795
lambda__H_1_ss          2.565
lambda__H_2_ss          1.590
lambda_ss               1.590
mc_ss                   1.000
r_ss                    0.055
w_ss                    1.613


In addition, the following parameter values were calibrated:
psi                     1.719


Alternatively, we can find the steady state by minimzing the sum of squared residuals of the steady state equations. This is nice because it lets the solver use  2nd derivate information to seek solution. It can be slower, though. For this simple model case, it doesn't matter -- both ways are fast and lead to the correct answer.

In [23]:
mod_log.steady_state(method='minimize')

Steady state found! Sum of squared residuals is 3.9651845564799144e-25


In [24]:
mod_log.print_steady_state()

A_ss                    1.000
C_ss                    0.629
Div_ss                 -0.000
I_ss                    0.166
K_ss                    4.733
L_ss                    0.670
N_ss                    0.330
TC_ss                  -0.795
U_ss                  -57.590
Y_ss                    0.795
lambda__H_1_ss          2.565
lambda__H_2_ss          1.590
lambda_ss               1.590
mc_ss                   1.000
r_ss                    0.055
w_ss                    1.613


In addition, the following parameter values were calibrated:
psi                     1.719


# CRRA Utility

The above analysis used a log intratemporal utility function:

$$u_t = \gamma \log C_t + \psi \log L_t$$

Another common utility function is CRRA, defined:

$$u_t = \frac{C_t^{1 - \sigma_C} - 1}{1 - \sigma_C} + \psi \frac{L_t^{1 - \sigma_L} - 1}{1 - \sigma_L}$$

This utility function leads to the follow first order conditions:

$$\begin{align}
\lambda_t &= C_t^{-\sigma_C}\\
\lambda_{2,t} &= \psi L_t^{-\sigma_L} \\
\lambda_{2,t} &= w_t \lambda_t \\
\frac{\lambda_t}{\lambda_{t+1}} &= \beta (1 - \delta + r_{t+1})
\end{align}$$

After simplification we get the demand curve for labor:

$$w_t = \frac{1}{\psi} L_t^{\sigma_L} C_t^{-\sigma_L}$$

And the Euler equation:

$$ \left ( \frac{C_{t+1}}{C_t} \right )^{\sigma_C} = \beta (1 - \delta + r_{t+1})$$

## Firm Problem

The firm's problem is unchanged, we have the following demand curves:

$$\begin{align} r_t &= \alpha mc_t \frac{Y_t}{K_{t-1}} \\
                w_t &= (1 - \alpha) mc_t \frac{Y_t}{N_t} \end{align}$$
                
And marginal cost:

$$mc_t = \frac{1}{A_t} \left (\frac{r_t}{\alpha} \right )^\alpha \left (\frac{w_t}{1 - \alpha} \right )^{1 - \alpha}$$

## Steady State

From the above system I derive the analytic steady state. Before beginning, I note that the model prices are real prices, and the overall price level of the economy is normalized to 1, such that:

$$P_{ss} = 1$$

The model assumes perfect competition, so that $mc_t = P_t = 1$.

$A_t$ is AR(1) in logs with no trend, so $A_{ss} = 1$.

### Price system

$$\begin{align} r_{ss} &= \frac{1}{\beta} - (1 - \delta) \\
                w_{ss} &= (1 - \alpha) \left (\frac{\alpha}{r_{ss}} \right)^{\frac{\alpha}{1 - \alpha}} \end{align}$$
                
### Factor Markets
#### Capital Markets
$$\begin{align}
I_{ss} &= \delta K_{ss} \\
K_{ss} &= \alpha \frac{Y_{ss}}{r_{ss}} \\
I_{ss} &= \alpha \delta \frac{Y_{ss}}{r_{ss}}
\end{align}$$

#### Labor Market

$$\begin{align}
L_{ss}^{\sigma_L} &= \psi w_{ss} C_{ss}^{\sigma_C} \\
1 &= N_{ss} + L_{ss} \\
N_{ss} &= (1 - \alpha) \frac{Y_{ss}}{w_{ss}} \\
C_{ss} &= \left (1 - (1 - \alpha) \frac{Y_{ss}}{w_{ss}} \right )^{\frac{\sigma_L}{\sigma_C}}\left(\psi w_{ss} \right)^{-\frac{1}{\sigma_C}}
\end{align}$$

### AS-AD Equlibrium

$$\begin{align}
Y_{ss} &= C_{ss} + I_{ss} \\
Y_{ss} &= \left (1 - (1 - \alpha) \frac{Y_{ss}}{w_{ss}} \right )^{\frac{\sigma_L}{\sigma_C}}\left(\psi w_{ss} \right)^{-\frac{1}{\sigma_C}} + \alpha \delta \frac{Y_{ss}}{r_{ss}}
\end{align}$$

At this point, it is clear that no analytic steady state is available. This is due to the linear $\left (1 - (1 - \alpha) \frac{Y_{ss}}{w_{ss}} \right )^{\frac{\sigma_L}{\sigma_C}}$, introduced by the leisure term inside the utility function. We will have to find the steady state via numerical methods.

In [25]:
mod_crra = ge.gEconModel('rbc_crra_utility.gcn')
mod_crra.steady_state()

Model Building Complete.
Found:
	14 equations
	14 variables
	The following "variables" were defined as constants and have been substituted away:
		mc_t
	1 stochastic shock
		 0 / 1 has a defined prior. 
	6 parameter
		 0 / 6 has a defined prior. 
	1 calibrating equation
	1 parameter to calibrate
 Model appears well defined and ready to proceed to solving.

Steady state found! Sum of squared residuals is 4.242047945474882e-23


In [26]:
mod_crra.print_steady_state()

A_ss                    1.000
C_ss                    0.629
Div_ss                 -0.000
I_ss                    0.166
K_ss                    4.733
L_ss                    0.670
N_ss                    0.330
TC_ss                  -0.795
U_ss                  -61.837
Y_ss                    0.795
lambda__H_1_ss          3.234
lambda_ss               2.004
r_ss                    0.055
w_ss                    1.613


In addition, the following parameter values were calibrated:
psi                     1.452


# Comparison of Steady States

In [27]:
def compare_steady_states(models, names, precision=3):
    all_keys = set()
    for model in models:
        all_keys = all_keys.union(set(list(model.steady_state_dict.keys())))
    
    all_calibration = set()
    for model in models:
        all_calibration = all_calibration.union(set(list(model.calib_param_dict.keys())))
    
    header = ' ' * 15 + ''.join([f'{name:>16}' for name in names])
    print(header)
    print('-' * len(header))

    for key in all_keys:
        line = f'{key:<15}'
        for model in models:
            if key in model.steady_state_dict.keys():
                value = '{:>15.{}f}'.format(model.steady_state_dict[key], precision)
            else:
                value = f'{"---":>15}'
            line += value 
        print(line)
    print('\n')
    print('Calibrated parameters')
    print('-'* 50)
    for key in all_calibration:
        line = f'{key:<15}'
        for model in models:
            if key in model.calib_param_dict.keys():
                value = '{:>15.{}f}'.format(model.calib_param_dict[key], precision)
            else:
                value = f'{"---":>15}'
            line += value 
        print(line)
    
    

Steady state values remain the same between the two specifications, but this is due to calibration of the $\psi$ parameter setting $N_{ss} = 0.33$ in both cases. 

In [28]:
compare_steady_states([mod_log, mod_crra], ['Log Utility', 'CRRA Utility'], 4)

                    Log Utility    CRRA Utility
-----------------------------------------------
A_ss                    1.0000         1.0000
w_ss                    1.6135         1.6135
K_ss                    4.7330         4.7330
lambda__H_2_ss          1.5897            ---
L_ss                    0.6700         0.6700
I_ss                    0.1657         0.1657
lambda__H_1_ss          2.5650         3.2340
N_ss                    0.3300         0.3300
lambda_ss               1.5897         2.0044
r_ss                    0.0554         0.0554
C_ss                    0.6290         0.6290
TC_ss                  -0.7947        -0.7947
Div_ss                 -0.0000        -0.0000
Y_ss                    0.7947         0.7947
mc_ss                   1.0000            ---
U_ss                  -57.5899       -61.8367


Calibrated parameters
--------------------------------------------------
psi                     1.7185         1.4518
